In [1]:
from inspect_ai import eval, Task, task
from inspect_ai.dataset import Sample
from inspect_ai.scorer import Score, Target, accuracy, scorer
from inspect_ai.solver import Plan, TaskState, generate, use_tools, basic_agent
from inspect_ai.tool import bash
from inspect_ai.util import sandbox

In [2]:
@scorer(metrics=[accuracy()])
def check_file_exists():
    """check if file exists"""
    async def score(state: TaskState, target: Target):
        try:
            output = await sandbox().exec('ls')            
            print(output)
            _ = await sandbox().read_file(target.text)
            exists = True
            
        except FileNotFoundError:
            exists = False
        return Score(value=1 if exists else 0)

    return score


In [3]:
agent = basic_agent(tools=[bash()])

In [4]:
@task
def hello_world_challenge() -> Task:
    return Task(
        dataset=[
            Sample(
                input=(
                    "Create a file called `hello-world.txt`. "
                    "When you have finished, submit the string 'submit'."  # Without this, the agent is liable to loop until it hits its limits.
                ),
                target="hello-world.txt",
            )
        ],
        # solver=[use_tools([bash()]), generate()],
        solver=agent,
        sandbox="docker",
        scorer=check_file_exists(),
    )

In [7]:
log = eval(hello_world_challenge,
     model="openai/gpt-4o-mini",
     )

[03/26/25 23:27:41] WARNING  Unexpected error executing docker:                                       prereqs.py:87

PrerequisiteError: ERROR: Docker sandbox environments require Docker Engine

Install: https://docs.docker.com/engine/install/

In [ ]:
dict = log.__dict__


In [ ]:
print(dict)

In [1]:
from inspect_ai.model import get_model

model = get_model("anthropic/claude-3-5-sonnet-latest")
response = await model.generate("Say hello")

In [2]:
response

ModelOutput(model='claude-3-5-sonnet-20241022', choices=[ChatCompletionChoice(message=ChatMessageAssistant(content=[ContentText(type='text', text='Hello! How are you today?')], source='generate', role='assistant', tool_calls=None), stop_reason='stop', logprobs=None)], usage=ModelUsage(input_tokens=9, output_tokens=10, total_tokens=19, input_tokens_cache_write=0, input_tokens_cache_read=0), time=0.792976800003089, metadata=None, error=None)